<a href="https://colab.research.google.com/github/AndresWV/AndresWV.github.io/blob/main/ProyectoWisdm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Librerias usadas**

In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

**1) Obtain**


1.1 Crear archivo sin puntos y comas para despues crear el df




In [15]:
!sed 's/;//' 'WISDM_ar_v1.1_raw.txt'  > 'wisdmHar.nosemi'


1.2 Definir cabecera para generar el df con los datos



In [30]:
columns = ['user','activity','timestamp','x','y','z']
wisdmHar = pd.read_csv('wisdmHar.nosemi',header=None,names=columns,sep=',').dropna()


**Entrega 02 - SCRUB**



Eliminar filas con datos faltantes


In [42]:
wisdmHar = wisdmHar.dropna() 

Agregar al DataFrame los nombres de las columnas(Esta en el 1.2)



Visualizar las 5 primeras filas del DataFrame (.head())


In [43]:
wisdmHar.head()

,user,activity,timestamp,x,y,z
0,33,Jogging,49105962326000,-0.694638,12.680544,0.503953
1,33,Jogging,49106062271000,5.012288,11.264028,0.953424
2,33,Jogging,49106112167000,4.903325,10.882658,-0.081722
3,33,Jogging,49106222305000,-0.612916,18.496431,3.023717
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164


Buscar datos nulos y eliminarlos si es que existen

In [45]:
nVacios = wisdmHar.isnull().sum().sum()
if nVacios>0:
  wisdmHar = wisdmHar.dropna()
else:
  print("No hay datos nulos!") 


No hay datos nulos!


Visualizar los tipo de datos a los que pertenece cada columna (.info())

In [46]:
wisdmHar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151665 entries, 0 to 151664
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user       151665 non-null  int64  
 1   activity   151665 non-null  object 
 2   timestamp  151665 non-null  int64  
 3   x          151665 non-null  float64
 4   y          151665 non-null  float64
 5   z          151665 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 8.1+ MB


Cambiar el tipo de datos de las variables predictoras a valores reales, para realizar la clasificación

In [47]:
wisdmHar = wisdmHar.astype({'x':'float','y':'float','z':'float'})

Imprimir los tipos de datos de las variables del DataFrame

In [48]:
wisdmHar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151665 entries, 0 to 151664
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user       151665 non-null  int64  
 1   activity   151665 non-null  object 
 2   timestamp  151665 non-null  int64  
 3   x          151665 non-null  float64
 4   y          151665 non-null  float64
 5   z          151665 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 8.1+ MB


Eliminar las columnas que no se utilizaran para la clasificación

In [49]:
wisdmHar = wisdmHar.drop(['user', 'timestamp'], axis=1)

Utilizar el comando value_counts para obtener un detalle de cuantas instancias se tienen por actividad. El objetivo consiste en identificar la presencia del balance o desbalance en las categorías a modelar

In [51]:
wisdmHar.value_counts('activity')

activity
Walking       60708
Jogging       56697
Downstairs    17778
Upstairs      16482
dtype: int64

Este proyecto considera datos con un desbalance de clases. Es necesario balancearlos para que los modelos operen de manera adecuada. Existen muchas maneras de realizar un balance, pero en este proyecto utilizaremos una estrategia sencilla e intuitiva. Simplemente debe identificar la actividad que contenga el número menor de instancias. Se debe realizar un muestreo para las otras actividades utilizando ese valor. De esta manera todas las clases tendrán la misma cantidad de instancias.

Establecer el número de instancias y dimensiones del data frame resultante (shape)

Volver a imprimir la cantidad de instancias por categoría para verificar que todo esté en orden. (value_counts)

In [ ]:
wisdmHar.value_counts('activity')

Volver a visualizar las cinco primeras líneas del DataFrame para revisar que todo esté en orden (head)

In [52]:
wisdmHar.head()

,activity,x,y,z
0,Jogging,-0.694638,12.680544,0.503953
1,Jogging,5.012288,11.264028,0.953424
2,Jogging,4.903325,10.882658,-0.081722
3,Jogging,-0.612916,18.496431,3.023717
4,Jogging,-1.184970,12.108489,7.205164


Separar las variables predictoras (X) de las ctegorias o etiquetas (y)

Estandarizar los datos numericos (StandardScaler)